In [1]:
import numpy as np
from gwbench import network
from gwbench.snr import scalar_product_freq_array

In [2]:
def get_network_response(inj_params, f_min=5., f_max=1024., d_f = 2**-4,
        network_spec = ['CE-40_C', 'CE-20_S', 'ET_ET1', 'ET_ET2', 'ET_ET3'], 
        approximant='IMRPhenomD', 
        deriv_symbs_string = 'Mc eta chi1z chi2z DL tc phic iota ra dec psi', 
        cond_num=1e25, calc_detector_responses=True, calc_derivs=True, calc_fisher=True, **kwargs):
    

    # initialize the network with the desired detectors
    net = network.Network(network_spec)

    # choose the desired waveform 
    wf_model_name = 'lal_bbh'
    # pass the chosen waveform to the network for initialization
    net.set_wf_vars(wf_model_name=wf_model_name, wf_other_var_dic = {'approximant': approximant})

    # pick the desired frequency range
    f = np.arange(f_min, f_max, d_f)

    # choose whether to take Earth's rotation into account
    use_rot = 0

    # pass all these variables to the network
    net.set_net_vars(
        f=f, inj_params=inj_params,
        deriv_symbs_string=deriv_symbs_string,
        use_rot=use_rot, **kwargs
        )

    # compute the WF polarizations
    net.calc_wf_polarizations()

    if not calc_detector_responses:
        return net
    
    if calc_derivs:
        # compute the WF polarizations and their derivatives
        net.calc_wf_polarizations_derivs_num()

    # setup antenna patterns, location phase factors, and PSDs
    net.setup_ant_pat_lpf_psds()

    # compute the detector responses
    net.calc_det_responses()

    if calc_derivs:
        # compute the detector responses and their derivatives
        net.calc_det_responses_derivs_num()

    # calculate the network and detector SNRs
    net.calc_snrs()

    if calc_fisher:
        # calculate the network and detector Fisher matrices, condition numbers,
        # covariance matrices, error estimates, and inversion errors
        net.calc_errors(cond_sup=cond_num)


    return net

In [3]:
# Compute Naiive biases
def cutler_vallisneri_overlap_vec(del_hf, delta_hf, psd, freqs, df=None):
    return np.array([ scalar_product_freq_array(del_hf[deriv], delta_hf, psd, freqs, df=df) for deriv in del_hf ])

inj_params = dict()
inj_params['Mc'] = 30.9
inj_params['eta'] = 0.247
inj_params['iota'] = np.pi/4
inj_params['chi1x'] = 0
inj_params['chi1y'] = 0
inj_params['chi1z'] = 0
inj_params['chi2x'] = 0
inj_params['chi2y'] = 0
inj_params['chi2z'] = 0
inj_params['DL'] = 475
inj_params['tc'] = 0
inj_params['phic'] = 0
inj_params['ra'] = np.pi/4
inj_params['dec'] = np.pi/4
inj_params['psi'] = np.pi/4
inj_params['gmst0'] = 0
inj_params['phiRef'] = 0

deriv_symbs_string = 'Mc eta DL tc phic iota ra dec psi'
deriv_variables = ['log_Mc', 'eta', 'log_DL', 'tc', 'phic', 'cos_iota', 'ra', 'cos_dec', 'psi']

d_f = 1/16
f_low = 10.
f_high = 2048.
network_spec = ['A+_H', 'A+_L', 'V+_V']

net_ap = get_network_response(inj_params=inj_params, 
    deriv_symbs_string=deriv_symbs_string,
    f_min=f_low, f_max=f_high, d_f=d_f,
    approximant='IMRPhenomD', network_spec=network_spec 
    ,conv_cos = ('iota','dec'),
    conv_log = ('Mc','DL')
    )

net_tr = get_network_response(inj_params=inj_params, 
    deriv_symbs_string=deriv_symbs_string,
    f_min=f_low, f_max=f_high, d_f=d_f,
    approximant='IMRPhenomXAS', network_spec=network_spec
    ,conv_cos = ('iota','dec'),
    conv_log = ('Mc','DL')
    )

overlap_vecs_network = np.zeros((len(net_ap.detectors), len(net_ap.deriv_variables)))

for d in range(len(net_ap.detectors)):
    ## set up initial waveforms
    h1 = net_tr.detectors[d].hf
    h2 = net_ap.detectors[d].hf
    f = net_ap.detectors[d].f
    Sn = net_ap.detectors[d].psd
    network_spec_d = [net_ap.detectors[d].det_key]

    ## Compute CV overlap vector for this detector
    delta_hf = net_tr.detectors[d].hf - net_ap.detectors[d].hf
    overlap_vecs_network[d] = cutler_vallisneri_overlap_vec(net_ap.detectors[d].del_hf, delta_hf, net_ap.detectors[d].psd, net_ap.detectors[d].f)

cv_bias = np.matmul(net_ap.cov, np.sum(overlap_vecs_network, axis=0))
cv_bias = cv_bias.tolist()[0]

2023-12-19 14:26:33,967 - Network - INFO : Polarizations calculated.
2023-12-19 14:26:33,968 - Network - INFO : Calculate numeric derivatives of polarizations.
2023-12-19 14:26:33,975 - Network - INFO : Polarizations calculated.


2023-12-19 14:26:38,002 - Network - INFO : Numeric derivatives of polarizations calculated.
2023-12-19 14:26:38,026 - Network - INFO : PSDs loaded.
2023-12-19 14:26:38,029 - Network - INFO : Antenna patterns and LPFs loaded.
2023-12-19 14:26:38,040 - Network - INFO : Detector responses calculated.
2023-12-19 14:26:38,041 - Network - INFO : Calculate numeric derivatives of detector responses.
2023-12-19 14:26:38,041 - Network - INFO :    A+_H
2023-12-19 14:26:41,047 - Network - INFO :    A+_L
2023-12-19 14:26:44,013 - Network - INFO :    V+_V
2023-12-19 14:26:46,989 - Network - INFO : Numeric derivatives of detector responses calculated.
2023-12-19 14:26:46,993 - Network - INFO : SNRs calculated.
2023-12-19 14:26:46,994 - Network - INFO : Calculate errors (Fisher & cov matrices).
2023-12-19 14:26:46,994 - Network - INFO :    A+_H
2023-12-19 14:26:47,075 - Network - INFO :    A+_L
2023-12-19 14:26:47,154 - Network - INFO :    V+_V
2023-12-19 14:26:47,257 - Network - INFO : Errors calcula

In [4]:
print(cv_bias)

[0.011927592817112753, 0.032435896414379606, 1.7396149342051785, -0.0024082189985945163, 0.6106673240979035, 0.2773988703078203, -0.0015799870002783933, 0.0009277569480614645, -0.15251242323563474]


In [5]:
arnab_bias = np.array([ 1.30937195e-02, 3.38632109e-02, 1.74124245e+00, -2.14910145e-03, 3.90542706e-01, 2.64436484e-01, -1.87088785e-03, 8.53396225e-04, -6.38689364e-02 ])

cv_bias = np.array(cv_bias)
for n, bias in enumerate(100*np.abs(arnab_bias-cv_bias)/arnab_bias):
    print(f"{deriv_variables[n]}: {bias:.3f}%")

log_Mc: 8.906%
eta: 4.215%
log_DL: 0.093%
tc: -12.057%
phic: 56.364%
cos_iota: 4.902%
ra: -15.549%
cos_dec: 8.714%
psi: -138.790%
